In [ ]:
import redis
import json
# Configuration for the Redis connection
redis_host = 'redis-service'  # Update with your Redis host
redis_port = 6379             # Default Redis port
redis_db = 0                  # Redis database number
stream_name = 'WATERBODIES'           # Name of the Redis stream
consumer_group = 'my-consumer-group'  # Consumer group name
consumer_name = 'my-consumer'         # Unique name for the consumer

# Connect to Redis
r = redis.Redis(host=redis_host, port=redis_port, db=redis_db)

streams = ['STREAM', 'WATERBODIES', "WATERBODIESF"] 

for stream in streams:
    # Create a consumer group if it doesn't already exist
    try:
        r.xgroup_create(stream_name, consumer_group, id='0', mkstream=True)
    except redis.exceptions.ResponseError as e:
        # Ignore the error if the group already exists
        if "BUSYGROUP Consumer Group name already exists" not in str(e):
            raise

# Read messages from the stream
def read_stream():
    try:
        # Fetch new messages from the consumer group
        streams = {
            "STREAM": '>',
            "WATERBODIES": '>',
            "WATERBODIESF": '>',
        }

        messages = r.xreadgroup(consumer_group, consumer_name, streams, count=10, block=5000)
        if messages:
            for stream, events in messages:
                for event_id, event_data in events:

                    event_data_str = {k.decode("utf-8"): v.decode("utf-8") for k, v in event_data.items()}
                    
                    print(f"{stream_name} - Event Id: {event_id.decode('utf-8')} Event Data: {event_data_str['message']}")
                    
                    # Acknowledge the message
                    r.xack(stream_name, consumer_group, event_id)

    except Exception as e:
        print(f"Error reading stream: {e}")

# Continuously read from the stream
while True:
    read_stream()


In [23]:
import redis
import json

# Configuration for the Redis connection
redis_host = 'redis-service'  # Update with your Redis host
redis_port = 6379             # Default Redis port
redis_db = 0                  # Redis database number
consumer_group = 'my-consumer-group'  # Consumer group name
consumer_name = 'my-consumer'         # Unique name for the consumer

# Connect to Redis
r = redis.Redis(host=redis_host, port=redis_port, db=redis_db)

# List of stream names
streams = ['STREAM', 'WATERBODIES', 'WATERBODIESFAILURE']

# Create a consumer group for each stream if it doesn't already exist
for stream in streams:
    try:
        r.xgroup_create(stream, consumer_group, id='0', mkstream=True)
    except redis.exceptions.ResponseError as e:
        # Ignore the error if the group already exists
        if "BUSYGROUP Consumer Group name already exists" not in str(e):
            raise

# Read messages from the streams
def read_stream():
    try:
        # Stream dictionary with '>' for new messages
        stream_dict = {stream: '>' for stream in streams}

        # Fetch new messages from the consumer group
        messages = r.xreadgroup(consumer_group, consumer_name, stream_dict, count=10, block=5000)
        if messages:
            for stream, events in messages:
                for event_id, event_data in events:
                    # Decode the event data
                    event_data_str = {k.decode("utf-8"): v.decode("utf-8") for k, v in event_data.items()}
                    
                    print(f"{stream.decode('utf-8')} - Event Id: {event_id.decode('utf-8')} Event Data: {event_data_str}")

                    # Acknowledge the message in the current stream
                    r.xack(stream, consumer_group, event_id)

    except Exception as e:
        print(f"Error reading stream: {e}")

# Continuously read from the streams
while True:
    read_stream()


STREAM - Event Id: 1729358212025-0 Event Data: {'subject': 'https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2D_10TFK_20210708_0_L2A', 'producer': 'project-a', 'href': 'https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2D_10TFK_20210708_0_L2A'}
STREAM - Event Id: 1729358213026-0 Event Data: {'subject': 'https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20210708_0_L2A', 'producer': 'project-a', 'href': 'https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20210708_0_L2A'}
STREAM - Event Id: 1729358214027-0 Event Data: {'subject': 'https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A', 'producer': 'project-a', 'href': 'https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A'}
STREAM - Event Id: 1729358215028-0 Event Data: {'subject': 'https://ea